# Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import copy

# Import .ply, segment plant, and export .asc for import into plantscan3d

In [4]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
    
# image 4

input_file = "W-23-6a-slow_custom_0power_4.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud
#pcd = pcd.voxel_down_sample(voxel_size=0.005)

print("Radius oulier removal")
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.05)
#display_inlier_outlier(pcd, ind)

pcd = pcd.select_by_index(ind)

from matplotlib import pyplot as plt 
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.08, min_points=300, print_progress=True))
    
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
#o3d.visualization.draw_geometries([pcd])

finalPoints = np.asarray(pcd.points)[np.where(labels==5)]

finalPcd_four = o3d.geometry.PointCloud()
finalPcd_four.points = o3d.utility.Vector3dVector(finalPoints)

#o3d.visualization.draw_geometries([finalPcd_four])

pcd_down_four = finalPcd_four
#points = np.asarray(pcd_down_four.points)
#y_threshold = -0.12
##mask = points[:,1] > y_threshold
#pcd_down_four.points = o3d.utility.Vector3dVector(points[mask]) # normals and colors are unchanged

o3d.visualization.draw_geometries([pcd_down_four])

np.savetxt("final_point_cloud.asc", np.asarray(pcd_down_four.points), fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)

Radius oulier removal
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 6
point cloud has 6 clusters


# Import skeleton from plantscan3d as .txt file (MTG)

In [10]:
# Loading in data
filename = 'image_4.txt' #put your filename here

# setting everything up nicely

my_array = pd.read_csv(filename,skiprows=3,sep='\t',usecols=[0,1,3,4,5],names=['Node','Parent','X','Y','Z'])
my_array['Parent'].loc[0] = 0
my_array['Parent'] = my_array['Parent'].astype('int')

# visualize

my_array

E:\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Node,Parent,X,Y,Z
0,2,0,-0.009948,-0.069728,0.038775
1,3,2,-0.009381,-0.063063,0.036603
2,4,3,-0.009509,-0.049572,0.028704
3,7,4,-0.008002,-0.035813,0.023481
4,13,7,-0.005734,-0.024126,0.017773
5,20,13,-0.011288,-0.009588,0.001509
6,21,13,0.005039,-0.012246,0.007310
7,22,3,-0.027297,-0.005240,0.012878
8,27,20,-0.003285,0.005474,-0.003666
9,30,27,0.000145,0.021746,-0.005362


# Function for calculating branch angles

In [14]:
# function for returning a list of angles

def branch_angles(my_array):
    
    parents = my_array['Parent'] # getting all of the parent nodes
    angles = [] # list of angles for final output
    tested = [] #list of parents already tested

    for parent in parents: # for each parent in the list
        children = my_array[my_array['Parent'] == parent] # retrieve rows of dataframe that list the parent node as their parent
        if len(children) >= 2 and parent not in tested: # if that's 2 or greater, it must be a branching point
            parent_to_child_vectors = [] # list of vectors for branch vectors
            tested.append(parent) # we're testing it now, so add this parent node to the tested list so we don't do it again
            dot_products = [] # making a list of dot products for later
            reference_node_number = my_array[my_array['Node'] == parent]['Parent']
            reference_node_number = reference_node_number.values[0] # we need to first find the node preceding the parent to make a baseline vector
            parent_node_coordinates = np.array([my_array[my_array['Node']==parent]['X'],my_array[my_array['Node']==parent]['Y'],my_array[my_array['Node']==parent]['Z']]) #get the coorddiinates of the parent node
            reference_node_coordinates = np.array([my_array[my_array['Node']==reference_node_number]['X'],my_array[my_array['Node']==reference_node_number]['Y'],my_array[my_array['Node']==reference_node_number]['Z']]) # get the coordinates of the reference node
            reference_to_parent_vector = [parent_node_coordinates - reference_node_coordinates] #subtract to get the vector
            # for each child node, retrieve child node coordinates, make a child vector, calculate intervening branch angle and append to a list to check later
            for i in range(len(children)): 
                child_node_number = children['Node'].iloc[i] 
                child_node_coordinates = np.array([my_array[my_array['Node']==child_node_number]['X'],my_array[my_array['Node']==child_node_number]['Y'],my_array[my_array['Node']==child_node_number]['Z']]) #getting child node coordinates
                parent_to_child_vector = [parent_node_coordinates - child_node_coordinates] # calculating parent to child (or branch) vector
                parent_to_child_vectors.append(parent_to_child_vector) #adding to the list
                
                # this section calculates the dot between each to "branch" vector with the reference vector, with the logic being that the 
                # highest dot product will correspond to the branch vector/reference vector with the highest similarity to one another.
                # We'll treat this as the "axis" we compare against in the proper branch angle calculations
                
                unit_vector_1 = reference_to_parent_vector / np.linalg.norm(reference_to_parent_vector)
                unit_vector_2 = parent_to_child_vector / np.linalg.norm(parent_to_child_vector)
                unit_vector_1 = unit_vector_1.tolist()
                unit_vector_2 = unit_vector_2.tolist()
                unit_vector_1 = [unit_vector_1[0][0][0],unit_vector_1[0][1][0],unit_vector_1[0][2][0]]
                unit_vector_2 = [unit_vector_2[0][0][0],unit_vector_2[0][1][0],unit_vector_2[0][2][0]]
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                dot_products.append(dot_product)
            max_index = np.argmax(dot_products) # finding which dot product is the higest and getting its index
            reference_vector = parent_to_child_vectors[max_index] # retrieving the parent_to_child vector that corresponds to that maximal dot product

            # now that we've established what we're comparing against, we go through each branch vector and compare to this reference (being sure to not report out the 0 degrees in the case where we're comparing the same things)
            for i in range(len(parent_to_child_vectors)): 
                unit_vector_1 = reference_vector / np.linalg.norm(reference_vector)
                unit_vector_2 = parent_to_child_vectors[i] / np.linalg.norm(parent_to_child_vectors[i])
                unit_vector_1 = unit_vector_1.tolist()
                unit_vector_2 = unit_vector_2.tolist()
                unit_vector_1 = [unit_vector_1[0][0][0],unit_vector_1[0][1][0],unit_vector_1[0][2][0]]
                unit_vector_2 = [unit_vector_2[0][0][0],unit_vector_2[0][1][0],unit_vector_2[0][2][0]]
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                
                # getting angle in radians, then converting to degrees
                angle_in_radians = np.arccos(dot_product)
                angle_in_degrees = np.degrees(angle_in_radians)
                if angle_in_degrees > 0.00001: #as long as it's not 0, add and report
                    angles.append(angle_in_degrees)
    return angles

# Retrieve angles from our previously loaded array

In [15]:
angles = branch_angles(my_array)

<ipython-input-14-2e2cfc78fd9e>:53: RuntimeWarning: invalid value encountered in arccos
  angle_in_radians = np.arccos(dot_product)


# Print angles

In [16]:
print(angles)

[17.43320150971781, 48.95453635268195, 30.214822992503684, 48.885947517986295, 56.76823402691461, 93.45827202064585]
